# Speed Skydiving Analysis and Scoring

## Environment setup

In [1]:
from ssscore import COURSE_START
from ssscore import COURSE_END
from ssscore import DEG_IN_RAD
from ssscore import VALID_MSL
from ssscore import YEAR_PATH_TAG

import dateutil.parser
import math
import os
import os.path
import shutil

import pandas as pd

import ssscore

## FlySight data sources

1. Copy the FlySight Tracks/YY-MM-dd directory of interest to the DATA_LAKE directory;
   the DATA_LAKE can also be an external mount, a Box or Dropbox share, anything that 
   can be mapped to a directory -- even a whole drive!
1. Make a list of all CSV files in the FlySight DATA_LAKE
1. Move the CSV to the DATA_SOURCE bucket directory

### Define the data lake and data source

![Data sources diagram](images/SSScoring-data-sources.png)
<a id="l_data-def"></a>

In [21]:
DATA_LAKE   = './data-lake'
DATA_SOURCE = './data-sources/ciurana'

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

moved ./data-lake/15-08-13.CSV --> ./data-sources/ciurana/15-08-13.CSV
moved ./data-lake/13-53-39.CSV --> ./data-sources/ciurana/13-53-39.CSV
moved ./data-lake/12-54-33.CSV --> ./data-sources/ciurana/12-54-33.CSV
moved ./data-lake/11-50-10.CSV --> ./data-sources/ciurana/11-50-10.CSV
moved ./data-lake/10-00-31.CSV --> ./data-sources/ciurana/10-00-31.CSV
moved ./data-lake/09-08-18.CSV --> ./data-sources/ciurana/09-08-18.CSV
moved 6 CSV files from data lake [./data-lake] to data source [./data-sources/ciurana]


## Top speed and pitch analysis on a single data file

User selects a valid FlySight data path and file name.  Call the `calculateSpeedAndPitch(fileName)` function with this file name.  The function produces:

* Classical mean speed calculated from all the discrete speeds sampled within the course
* USPA calculation ds/dt where dt<sub>competitionTime</sub> = t<sub>end</sub>-t<sub>start</sub>,
  and s<sub>courseLength</sub> = |s<sub>start</sub>-s<sub>end</sub>| for the first and last heights
  reported within the course window.
* Flight pitch in degrees at max speed
* Max speed
* Min speed

All speeds are reported in km/h.  Michael Cooper, USPA National Speed Skydiving Championship judged, explained
the "USPA calculation" as straight ds/dt in an email message from 04.Sep.2018.

### Explanation of the code in calculateSpeedAndPitchFrom(fileName)

1. Discard all source entries outside of the 2,700 to 1,700 m AGL course
1. Calculate the mean speed for the current jump (actual sum(N)/N and ds/dt for s0, s1
1. Resolve the max speed and pitch within the course
1. Return the results in a Series object, for later inclusion in a data frame with results from multiple jumps

### Specify the FlySight data file to analyze

In [4]:
DATA_SOURCE        = './data-sources/ciurana'
FLYSIGHT_DATA_FILE = '10-00-31.CSV'

In [5]:
def _discardDataOutsideCourse(fullFlightData):
    height          = fullFlightData['hMSL', '(m)']
    descentVelocity = fullFlightData['velD', '(m/s)']
    
    return fullFlightData[(height <= COURSE_START) & (height >= COURSE_END) & (descentVelocity >= 0.0)]


def _calculateCourseSpeedUsing(flightData):
    """
    Returns course speed as ds/dt, and competitionTime in seconds
    """
    # TODO: figure out why pd.to_datetime() barfs
    startTime = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[0])
    endTime   = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[-1])

    startCourse = flightData['hMSL', '(m)'].values[0]
    endCourse   = flightData['hMSL', '(m)'].values[-1]

    competitionTime = (endTime-startTime).total_seconds()
    courseLength    = abs(endCourse-startCourse)

    return courseLength/competitionTime, competitionTime


def maxHorizontalSpeedFrom(flightData, maxVerticalSpeed):
    velN = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velN', '(m/s)']
    velE = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velE', '(m/s)']
    
    return math.sqrt(velN**2+velE**2)   # R vector


def calculateSpeedAndPitchFor(fileName):
    """
    Accepts a file name to a FlySight data file.
    
    Returns a Series with the results of a speed skydiving jump.
    """
    flightData  = _discardDataOutsideCourse(pd.read_csv(fileName, header = [0, 1]))
    sampledVelD = flightData['velD', '(m/s)'].mean()
    courseVelD, \
    courseTime  = _calculateCourseSpeedUsing(flightData)
    maxSpeed    = flightData['velD', '(m/s)'].max()
    minSpeed    = flightData['velD', '(m/s)'].min()
    
    pitchR       = math.atan(maxSpeed/maxHorizontalSpeedFrom(flightData, maxSpeed))
    maxSpeedTime = flightData[flightData['velD', '(m/s)'] == maxSpeed]['time', 'Unnamed: 0_level_1'].values[0]
    
    skydiveResults = pd.Series(
                        [
                            fileName,
                            maxSpeedTime,
                            3.6*sampledVelD,  # km/h; 3,600 seconds, 1,000 meters
                            3.6*courseVelD,
                            pitchR/DEG_IN_RAD,
                            3.6*maxSpeed,
                            3.6*minSpeed,
                            courseTime,
                        ],
                        [
                            'fileName',
                            'maxSpeedTime',
                            'sampledSpeed',
                            'courseSpeed',
                            'pitch',
                            'maxSpeed',
                            'minSpeed',
                            'courseTime',
                        ])

    return skydiveResults


calculateSpeedAndPitchFor(os.path.join(DATA_SOURCE, FLYSIGHT_DATA_FILE))

fileName        ./data-sources/ciurana/10-00-31.CSV
maxSpeedTime                2018-12-22T18:23:21.80Z
sampledSpeed                                359.618
courseSpeed                                 361.974
pitch                                       69.8803
maxSpeed                                     391.68
minSpeed                                        315
courseTime                                      9.6
dtype: object

## Listing FlySight data files with valid data

This set up generates a list of FlySight data files ready for analysis.  It discards any warm up FlySight data files, those that show no significant changes in elevation MSL across the complete data set.

The list generator uses the `DATA_SOURCE` global variable.  [Change the value of `DATA_SOURCE`](#l_data-def) if necessary.

1. Generate the list of available data files
1. Discard FlySight warm up files by rejecting files without jump data (test:  minimal altitude changes)

![Scoring FlySight files list generation diagram](images/SSScoring-list-scoring-files.png)

In [6]:
def listDataFilesIn(bucketPath):
    """
    Generate a sorted list of files available in a given bucketPath.
    Files names appear in reverse lexicographical order.
    """

    filesList = pd.Series([os.path.join(bucketPath, fileName) for fileName in sorted(os.listdir(bucketPath), reverse = True) if '.CSV' in fileName])
    
    return filesList;

In [7]:
def _hasValidJumpData(fileName):
    flightData = pd.read_csv(fileName, header = [0, 1])['hMSL', '(m)']
    
    return flightData.std() >= VALID_MSL

In [8]:
def selectValidFlySightFilesFrom(dataFiles):
    included = dataFiles.apply(_hasValidJumpData)
    
    return pd.Series(dataFiles)[included]

In [22]:
dataFiles = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE))
dataFiles

0    ./data-sources/ciurana/15-08-13.CSV
1    ./data-sources/ciurana/13-53-39.CSV
2    ./data-sources/ciurana/12-54-33.CSV
3    ./data-sources/ciurana/11-50-10.CSV
4    ./data-sources/ciurana/10-00-31.CSV
dtype: object

## Top speed and pitch analysis on all the data in a bucket

Takes all the FlySight files in a bucket, detects the ones with valid data, and runs performance analysis over them.  Packs all the results in a data frame, then calculates:

* Average speed
* Max average speed

_Future versions may adhere to USPA / ISSA judging rules for analysis._

### Define the data source

In [10]:
DATA_SOURCE = './data-sources/ciurana'

### Analyze all files in the bucket

In [23]:
allCompetitionJumps = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE)).apply(calculateSpeedAndPitchFor)
allCompetitionJumps

,fileName,maxSpeedTime,sampledSpeed,courseSpeed,pitch,maxSpeed,minSpeed,courseTime
0,./data-sources/ciurana/15-08-13.CSV,2018-12-22T23:29:45.00Z,364.618286,366.203250,73.491872,398.232,313.488,9.6
1,./data-sources/ciurana/13-53-39.CSV,2018-12-22T22:22:33.80Z,326.802000,329.224755,68.195275,369.756,293.004,10.6
2,./data-sources/ciurana/12-54-33.CSV,2018-12-22T21:20:46.00Z,345.857538,346.812706,68.684885,379.440,322.848,10.2
3,./data-sources/ciurana/11-50-10.CSV,2018-12-22T20:16:28.40Z,345.806308,347.756118,69.012308,380.016,304.488,10.2
4,./data-sources/ciurana/10-00-31.CSV,2018-12-22T18:23:21.80Z,359.617959,361.974000,69.880285,391.680,315.000,9.6


### Summary of results